Training, Validation and testing dataset were labeled by batch manually

In [5]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import CT_registration_diffusion.functions_collection as ff

In [6]:
main_path = '/host/d/Data/4DCT'
datasets = ff.find_all_target_files(['DIR*','Pop*'],main_path)

# select batch


In [7]:
def make_batches(n_samples, n_batches=5):
  
    batch_list = []
    batch_size = n_samples // n_batches
    remainder = n_samples % n_batches

    for i in range(n_batches):
        size = batch_size + (1 if i == n_batches - 1 and remainder > 0 else 0)
        batch_list.extend([i] * size)
    return batch_list

In [8]:
results = []
for dataset_num in range(len(datasets)):
    dataset_path = datasets[dataset_num]
    dataset_ID = os.path.basename(dataset_path)
    
    cases = ff.sort_timeframe(ff.find_all_target_files(['*'], dataset_path),0, 'e','')

    num_of_cases = len(cases)
    batch_list = make_batches(num_of_cases, n_batches=5)

    for n in range(0,num_of_cases):
        batch = batch_list[n]
        case_path = cases[n]

        case_id = os.path.basename(case_path)
        results.append([batch, dataset_ID, case_id])
df = pd.DataFrame(results, columns=['batch', 'dataset_ID', 'case_ID'])
ff.make_folder([os.path.join(main_path,'Patient_lists')])
df.to_excel(os.path.join(main_path,'Patient_lists','batch_list.xlsx'), index=False)

# Make patient list for train and testing

In [9]:
batch_list = pd.read_excel(os.path.join(main_path,'Patient_lists','batch_list.xlsx'))
results = []
for dataset_num in range(0, len(datasets)):
    batch = 0
    dataset_path = datasets[dataset_num]
    print(dataset_path)

    dataset_ID = os.path.basename(dataset_path)

    cases = ff.sort_timeframe(ff.find_all_target_files(['*'], dataset_path),0, 'e','')

    for case_num in range(0, len(cases)):
        case_path = cases[case_num]

        case_ID = os.path.basename(case_path)

        row = batch_list[(batch_list['dataset_ID'] == dataset_ID) & (batch_list['case_ID'] == case_ID)]
        if not row.empty:
            batch = row['batch'].values[0]

        image_folder = os.path.join(main_path, dataset_ID, case_ID, 'cropped_image')

        results.append([batch, dataset_ID, case_ID, image_folder])
   
results_df = pd.DataFrame(results, columns = ['batch', 'dataset_ID', 'case_ID', 'image_folder'])
ff.make_folder([os.path.join(main_path,'Patient_lists')])
results_df.to_excel(os.path.join(main_path,'Patient_lists','ct_list.xlsx'), index = False)
    

/host/d/Data/4DCT/DIR_LAB
/host/d/Data/4DCT/Popi
